<h1 align="center">Quantitative Asset Management Write Up (with code)</h1> 

## Bryan Marty, CFA


1. Construct the value-weighted market return using CRSP data,1
replicating the market return
time series available in Kenneth French website.2 Also calculate the equal-weighted market return,
and the lagged total market capitalization. Your output should be from January 1926 to December
2021, at a monthly frequency.






### Methodology for Question 1


### Input
<table>
    <tr>
        <td>
\begin{eqnarray}
     
            
     Variable Name  & VariableType\\
     ----------- & ------------\\
     PERMNO &  datetime\\
     SHRCD & integer\\
     EXCHCD & integer\\
     RET & float\\
     DLRET & float\\
     PRC & float\\
     SHROUT & integer\\
    \end{eqnarray}
     
        <td>
    </tr>
</table>
            


### Stock Universe:

Following the methodology explained in class, I restrict this sample to common shares of domestic and international companies whose common shares trade on NYSE, American Stock Exchange and the Nasdaq through share-codes (10, 11, 12) and with exchange codes (1, 2, and 3) respectively to derive the above input dataframe 



 

In [14]:
##Load and Save Data in .pkl format
import os
os.getcwd()
import pandas as pd
import numpy as np
import datetime as dt
import psycopg2
import wrds
from random import *
from dateutil.relativedelta import * 
from pandas.tseries.offsets import * 
from scipy import stats
import matplotlib.pyplot as plt 
import requests
import pandas_datareader
import datetime
import sqlalchemy as sa
import datetime as dt
from datetime import timedelta
from dateutil.parser import parse
from scipy.stats import kurtosis, skew

# WRDS login information

data_folder =  r"C:\Users\user\Downloads"   # folder where I’ll keep data
id_wrds = 'blacksheep'  # add your own wrds id


In [15]:
load_data = False
if load_data:
    print('Load Data')
else:
    pd.read_pickle(data_folder + 'mcrsp_raw.pkl')
    pd.read_pickle(data_folder + 'dlret_raw.pkl')
    print('Move onto CRSP RAW Returns Cleaning Block')

Move onto CRSP RAW Returns Cleaning Block


In [ ]:

# It's useful to save your password: run the following
conn = wrds.Connection(wrds_username=id_wrds)
conn.create_pgpass_file()
conn.close()

# It's useful to save your password: run the following
conn = wrds.Connection(wrds_username=id_wrds)
conn.create_pgpass_file()
conn.close()


# Step 2: Load CRSP returns (monthly)
conn = wrds.Connection(wrds_username=id_wrds)
mcrsp_raw = conn.raw_sql("""
                      select a.permno, a.date, a.ret, b.ticker, a.shrout, b.shrcd, b.exchcd, a.prc
                      from crspq.msf as a
                      left join crspq.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      """)
conn.close()
mcrsp_raw.to_pickle(data_folder + 'mcrsp_raw.pkl')



# Step 3: Load CRSP Deslisting returns (monthly)
conn = wrds.Connection(wrds_username=id_wrds)
dlret_raw = conn.raw_sql("""
                      select a.permno, a.dlstdt, a.dlret, b.ticker, b.shrcd, b.exchcd
                      from crspq.msedelist as a
                      left join crspq.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.dlstdt
                      and a.dlstdt<=b.nameendt
                      """)
conn.close()
dlret_raw.to_pickle(data_folder + 'dlret_raw.pkl')  # ~254 KB


 
### Computation of Value-Weighted Market Returns

An additional column I called "me" is created for market cap from multiplying shares outstanding field ('shrout') and price ('prc').  We have used the absolute value of Price field since CRSP data can show negative values from price at times.  This is a result of CRSP data indicating accurate closing price information is not available for that day (and thus some approximation is used).  

Understand that the 'me' variable gives us the market capitalization of a particular stock during the end of the month in a particular year.





In [16]:
#CRSP Raw Returns - Data Cleaning
mcrsp_raw = pd.read_pickle(data_folder + 'mcrsp_raw.pkl')
mcrsp_raw['permno'] = mcrsp_raw['permno'].astype(int)
mcrsp_raw['date'] = pd.to_datetime(mcrsp_raw['date'], format = '%Y-%m-%d', errors = 'ignore')
mcrsp_raw = mcrsp_raw.sort_values(by=['ticker', 'date']).reset_index(drop=True).copy()
mcrsp_raw['me'] = mcrsp_raw['prc'].abs() * mcrsp_raw['shrout'] * 1e-6



#CRSP Raw Delisting Returns - Data Cleaning
dlret_raw = pd.read_pickle(data_folder + 'dlret_raw.pkl')
dlret_raw['permno'] = dlret_raw['permno'].astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'])
dlret_raw = dlret_raw.rename(columns = {"dlstdt": "date", "ticker": "dlticker"}).copy()
dlret_raw.head()


#Merged data - Initial cleaning
mcrsp = mcrsp_raw.merge(dlret_raw, how = 'outer', on = ['date', 'permno'])
mcrsp['shrcd'] = np.where(mcrsp['shrcd_x'].isna(), mcrsp['shrcd_y'], mcrsp['shrcd_x'])
mcrsp['exchcd'] = np.where(mcrsp['exchcd_x'].isna(), mcrsp['exchcd_y'], mcrsp['exchcd_x'])
mcrsp.drop('shrcd_x', axis = 1, inplace=True)
mcrsp.drop('shrcd_y', axis = 1, inplace=True)
mcrsp.drop('exchcd_x', axis = 1, inplace=True)
mcrsp.drop('exchcd_y', axis = 1, inplace=True)
mcrsp = mcrsp[['date', 'permno', 'ret', 'dlret', 'shrcd', 'exchcd', 'shrout', 'prc']].sort_values(by=['permno', 'date']).reset_index(drop=True).copy()
del mcrsp_raw, dlret_raw

mcrsp.head()

,date,permno,ret,dlret,shrcd,exchcd,shrout,prc
0,1985-12-31,10000,NaN,NaN,NaN,NaN,NaN,NaN
1,1986-01-31,10000,NaN,NaN,10.0,3.0,3680.0,-4.3750
2,1986-02-28,10000,-0.257143,NaN,10.0,3.0,3680.0,-3.2500
3,1986-03-31,10000,0.365385,NaN,10.0,3.0,3680.0,-4.4375
4,1986-04-30,10000,-0.098592,NaN,10.0,3.0,3793.0,-4.0000


## Function walkthrough:

### Delisting Return:  

For the first part, the return calculations both holding period and delisting period returns are calculated in the following manner from the class slides:

 ### $ r^{h}_{i,t} =  Return$
   if $r^{d}_{i,t}$ is missing
 ### $ r^{d}_{i,t} = Return$
   if $r^{h}_{i,t}$ is missing
 ### $(r^{h}_{i,t} + 1)(1 + r^{d}_{i,t}) -1 =  Return$  
   if both returns are not missing.
 
### DataFrame Variables explained

Next, columns year, date, lagged_mv, and market equity are re-computed and filtered to be added later to the final output.  It's important to see how all vectors are sorted by year and date to get the estimate for each month rolling forward. In particular, two new columns are created and split from date column into year and month.  The dataframe with my "me" column is created where the dataframe is grouped by year and month and the MKTEquity is added for all stocks in a particular given month.  This gives us the sum of market cap of all the stocks in a particular month.

Understand, we need weights of stocks to be computed to know what the portfolio weights (via mktvalue) are in a particular month.  This is obtained by dividing the mktcap of a single stock by the the total makret cap available in a month.  Since we use these weights in subsequent month, we need to lag these weights by one month on the data that is grouped by 'permno'.  Now we just remove nan values in wt and ret vectors 
and now we can compute value returns by month.

### Missing Returns:

If at the end of my work there are still missing returns I just drop those returns from consideration.

### Value Weighted

vwretd are computed in a particular month by multiplying weight and return ret.  The data is grouped by year and month and the vwretd values are all added to find the value weighted return in a particular monnth

   

### Equal Weighted

To calculate equal weighted return, we just group the data by year and month and just take the mean return of all stocks in that particular month. 


### Sample Period

We have considered a sample from January 1926 to December 2021 for all the calculations. 

### Definition of Portfolio Weights:

The portfolio weights help us understand the impact of the returns of each stock in the portfolio.  For example , as the weight of one stock increases in the portfolio, the effect that the stocks return now has on the proftolio increases.  This situation reverses in the opposite case.

### Here we observe the Value Weight:
### $$ w^{mkt}_{i,t} = \frac{me_{i,t-1}}{\sum^{n}_{i=1}me_{i,t-1}}$$

### Here we observe the Equal Weight:
### $$ w^{mkt}_{i,t} = 1/\sum^{N}_{i=1}(n_{i} Stocks)$$

In [21]:
# METHOD1: PS1_Q1 as prescribed in the question
def Q1(mcrsp):
    mcrsp['ret'] = np.where(mcrsp['ret'].notna() & mcrsp['dlret'].notna(), (1+mcrsp['ret'])*(1+mcrsp['dlret']) - 1, mcrsp['ret'])
    mcrsp['ret'] = np.where(mcrsp['ret'].isna() & mcrsp['dlret'].notna(), mcrsp['dlret'], mcrsp['ret'])
    mcrsp = mcrsp[mcrsp['exchcd'].isin([1,2,3]) & mcrsp['shrcd'].isin([10,11,12])]

    data = mcrsp.copy()
    data['year'] = pd.DatetimeIndex(mcrsp['date']).year
    data['month'] = pd.DatetimeIndex(mcrsp['date']).month
    data = data.sort_values(by=['year','month']).reset_index(drop=True)
    data["me"] = data['prc'].abs() * data['shrout'] * 1e-6

    temp = data[['me', 'year', 'month']].groupby(['year', 'month']).sum()
    temp = temp.rename(columns={"me": "Lagged_mv"})

    data = data.merge(temp, how='outer', on=['year', 'month'])
    data = data.sort_values(['permno', 'year', 'month'])
    data['wt'] = data['me']/data['Lagged_mv']
    #oh that's slick below
    data['wt'] = data.groupby(['permno'])['wt'].transform(lambda s: s.shift(1))
    data = data[data['wt'].notna()].copy()
    data = data[data['ret'].notna()].copy()
    data['Stock_VW_Ret'] = data['wt'] * data['ret']
    temp = temp.reset_index()[1:].reset_index(drop=True)
    vwretd = (data.groupby(['year','month'])['Stock_VW_Ret'].sum()).to_frame().reset_index()
    eqretd = (data.groupby(['year','month'])['ret'].mean()).to_frame().reset_index()
    output = pd.concat([vwretd, eqretd['ret'], temp['Lagged_mv']], axis = 1)
    output = output.rename(columns={"ret": "Stock_EW_Ret"})
    del temp, vwretd, eqretd, data
    return output


# OUTPUT 

 I'm using vwretd = my stock_vw_RET fyi. 

<table>
    <tr>
        <td>
\begin{eqnarray}
     Variable Name  & VariableType & Variable Description\\
     ----------- & ------------ & -------\\
     Year &  Integer & Year &\\
     Month & integer & Month\\
     Stock lag MV & float & Total Mkt Value Previous Month (mil)\\
     Stock EW Ret & float & Equal Weighted Returns\\
     Stock VW Ret & float & Value Weighted Returns\\
\end{eqnarray}    
        <td>
    </tr>
</table>

In [22]:
final = Q1(mcrsp)
final.head()

,year,month,Stock_VW_Ret,Stock_EW_Ret,Lagged_mv
0,1926,1,0.000541,0.023559,27.542032
1,1926,2,-0.033035,-0.053624,26.680178
2,1926,3,-0.063956,-0.092309,25.012447
3,1926,4,0.036679,0.032482,25.811801
4,1926,5,0.013417,0.005617,26.112447


# Question 2 Write UP

2. Using the risk-free rate of return from French’s website3
, report the following moments of
the market excess returns for both time series (4 decimal digits): annualized return, annualized
volatility, annualized Sharpe ratio, skewness, and excess kurtosis. You should be comparing
between July 1926 to December 2021, at a monthly frequency.

– Inputs
* dataframe Monthly CRSP Stocks, the output of PS1 Q1
* dataframe FF mkt, with columns

– Output
* 5×2 numeric matrix (dataframe). Rows: Annualized Mean, Annualized Standard
Deviation, Annualized Sharpe Ratio, Skewness, and Excess Kurtosis. Columns:
Estimated FF Market Excess Return, Actual FF Market Excess Return.

In [25]:
#returns for value and equal weighted portfolios and market value
#returns for value and equal weighted portfolios and market value
final['tempvwret'] = final['Stock_VW_Ret'] + 1
final['cumulative-vwret'] = final.tempvwret.cumprod()
final.drop('tempvwret', axis=1, inplace=True)
final['tempewret'] = final['Stock_EW_Ret'] + 1
final['cumulative'] = final.tempewret.cumprod()
final.drop('tempewret', axis=1, inplace=True)




### Importation and Cleaning Scripts

### Reading from CSVs downloaded from Ken French's website

As the dates from the Ken French website CSVs are in the format 201008, we need to write a function to parse this data. We show that parsing a input string like 192607 is parsed correctly. There were a few bugs in the professors code we needed to adjust for. 


In [26]:
# Fama French - Data Cleaning
ff_file = r"C:\Users\user\Documents\qam\F-F_Research_Data_Factors.CSV"
FF_mkt = pd.read_csv(ff_file)
FF_mkt['Date'] = pd.to_datetime(FF_mkt['Date'], format = '%Y%m', errors = 'ignore')
FF = FF_mkt.copy()
FF['year'] = pd.DatetimeIndex(FF_mkt['Date']).year
FF['month'] = pd.DatetimeIndex(FF_mkt['Date']).month
FF.drop('Date', axis = 1, inplace=True)
FF[['Mkt-RF', 'SMB', 'HML', 'RF']] = FF[['Mkt-RF', 'SMB', 'HML', 'RF']] / 100


------------------------------
### ANALYSIS

The following summary statistics to be presented below.  The reported statistics are estimaged FF Market excess return that is calculated in Question 1 and actual market excess return taken from Fama/French 3 factor dataset.  Initial data cleaning is dobe by splitting 'year' and 'month' columns from the date column.  'Mkt-RF', 'SMB, 'HML', 'RF' are divided by 100 since the returns are in the percentage in the Fama/French data.  Understand that the Fama French data and output from Question 1 is used as inputs for PS1_Q2 method. 

The questions statistics are calculated as follows:

### 1 Sample Period:  
    Monthly data from July 1926 to December 2020 is used for Fama/French calculations.j
### 2 Excess Returns:  
    "Mkt-RF' column is used for excess return from FF data (actual) and difference of 'vwretd' and 'RF' is used for excess return from data in Question 1 (estimated via WRDS CRSP).
### 3 Annualized Mean:  

    The mean excess return is calculated from both monthly time series and multiplied by 12 to get annualized means
### $ \mu_{annualized} = 12  \mu_{monthly}$

### 4 Annualized Standard Deviation:  

The standard deviation is calculated in both the series and multiplied by the square root of 12 for annualization.

### $\sigma_{annualized} = \sqrt{12}*\sigma_{monthly}$

### 5 Annualized Sharpe Ratio:  

The annualized means are divided by the annualized standard deviation to obtain the annualized Sharpe Ratio. 

### $ SR_{annualized} = \frac{\mu_{annualized}}{\sigma_{annualized}}$

### 6 Excess Skewness: 

The skewness is calculated from both series directly for the excess returns.  THRID MOMENT !



### $ \gamma_{1} = \mathbb{E}[\frac{(X - \mu)}{\sigma}^{3}]$

### 7 Kurtosis:  

The Kurtosis is calculated from both series directly from the excess returns.

### $\gamma_{1} = \frac{\sum^{n}_{i}(X_{i} - X^{hat})^{4}}{N}$


In [28]:

# METHOD 2: PS1_Q2 as prescribed in the question
def Q2(Monthly_CRSP_Stocks, FF):
    Monthly_CRSP_Stocks = Monthly_CRSP_Stocks.merge(FF, how='inner', on=['year', 'month'])
    Monthly_CRSP_Stocks['estimated'] = Monthly_CRSP_Stocks['Stock_VW_Ret'] - Monthly_CRSP_Stocks['RF']

    estimated_mean = 12 * Monthly_CRSP_Stocks['estimated'].mean()
    estimated_std = np.sqrt(12) * Monthly_CRSP_Stocks['estimated'].std()
    estimated_sr = estimated_mean/estimated_std
    estimated_kurtosis = kurtosis(Monthly_CRSP_Stocks['estimated'])
    estimated_skew = skew(Monthly_CRSP_Stocks['estimated'])

    actual_mean = 12 * Monthly_CRSP_Stocks['Mkt-RF'].mean()
    actual_std = np.sqrt(12) * Monthly_CRSP_Stocks['Mkt-RF'].std()
    actual_sr = actual_mean/actual_std
    actual_kurtosis = kurtosis(Monthly_CRSP_Stocks['Mkt-RF'])
    actual_skew = skew(Monthly_CRSP_Stocks['Mkt-RF'])

    result = pd.DataFrame({'Estimated FF Market Excess Return': [round(estimated_mean,4), round(estimated_std,4), round(estimated_sr,4), round(estimated_skew,4), round(estimated_kurtosis,4)], 'Actual FF Market Excess Return': [ round(actual_mean,4), round(actual_std,4), round(actual_sr,4), round(actual_skew,4), round(actual_kurtosis,4)]}, ["Annualized Mean", "Annualized Standard Deviation", "Annualized Sharpe Ratio", "Skewness", "Excess Kurtosis"])

    return result


k = Q2(final, FF)
k


,Estimated FF Market Excess Return,Actual FF Market Excess Return
Annualized Mean,0.0838,0.0832
Annualized Standard Deviation,0.1853,0.1848
Annualized Sharpe Ratio,0.4521,0.4502
Skewness,0.1536,0.1632
Excess Kurtosis,7.6136,7.6097


# Question 3

3. Report (up to 8 decimal digits) the correlation between your time series and French’s time
series, and the maximum absolute difference between the two time series. It is zero? If not, justify
whether the difference is economically negligible or not. What are the reasons a nonzero difference?
You should be comparing between July 1926 to July 2021, at a monthly frequency.


Suggested function: PS1 Q3
– Inputs
* dataframe Monthly CRSP Stocks, the output of PS1 Q1
* dataframe FF mkt as described in PS1 Q2
– Output
* Vector of length two. Correlation between time series and maximum absolute
difference between two time series.


In [29]:
# Method 3: PS1_Q3 as prescribed in the question
def Q3(Monthly_CRSP_Stocks, FF):
    Monthly_CRSP_Stocks = Monthly_CRSP_Stocks.merge(FF, how='inner', on=['year', 'month'])
    Monthly_CRSP_Stocks['WRDS_EST'] = Monthly_CRSP_Stocks['Stock_VW_Ret'] - Monthly_CRSP_Stocks['RF']

    correlation = Monthly_CRSP_Stocks['Mkt-RF'].corr(Monthly_CRSP_Stocks['WRDS_EST'])
    max_diff = max(abs(Monthly_CRSP_Stocks['WRDS_EST'] - Monthly_CRSP_Stocks['Mkt-RF']))

    result = [round(correlation,8), round(max_diff,8)]

    return result



q = Q3(final, FF)


answer = pd.DataFrame({'Result': [q[0], q[1]]}, ["Correlation between Time Series", "Maximum absolute difference b/w two time series"])
answer

,Result
Correlation between Time Series,0.999555
Maximum absolute difference b/w two time series,0.015394


### Analysis:

Here we simply take the wrds mkt return estimate and compare it directly with the FamaFrench Mkt-rf equity risk premium to observe the correlation. 
The "Wrds Est" is simply the value weighted return subtracted from the FF rf rate and a pearson correlation between FF Mkt-rf and WRDS_EST vector is taken.  We also find the absolute greatest deviation from the two series and report that as well. 